# Lab 4 - Reinforcement Learning

## Read details of the environment & model from a text file
Read: 

- Width, Height
- Noise: probability of not going in the intended direction (then divided by two for two unintentional neighbor directions)
- Immediate rewards of non-goal states
- Terminal (goal) states: their locations and rewards
- Internal Walls: locations


Task1 :
- Reading and parsing the grid1.txt file to extract grid details.
  
- Creating transition and reward matrices for the Grid World environment described in grid1.txt.
  
- Defining two additional grid worlds with unique configurations.
  
- Creating transition and reward matrices for these additional grids.

In [6]:
import numpy as np

# Function to convert (x, y) coordinates to a state index
def xy_to_state(x, y, width):
    return y * width + x

# Function to create transition and reward matrices for a given grid configuration
def create_matrices(width, height, terminal_states, walls, immediate_reward):
    num_states = width * height
    num_actions = 4  # Up, Down, Left, Right
    transition_matrix = np.zeros((num_states, num_actions, num_states))
    reward_matrix = np.zeros((num_states, num_actions, num_states)) + immediate_reward

    for x in range(width):
        for y in range(height):
            if (x, y) in walls or (x, y) in terminal_states:
                continue

            current_state = xy_to_state(x, y, width)

            for action, (dx, dy) in enumerate([(-1, 0), (1, 0), (0, -1), (0, 1)]):
                next_state = (x + dx, y + dy)

                if 0 <= next_state[0] < width and 0 <= next_state[1] < height and next_state not in walls:
                    next_state_idx = xy_to_state(*next_state, width)
                    transition_matrix[current_state, action, next_state_idx] += 0.8

                for side_dx, side_dy in [(dy, dx), (-dy, -dx)]:
                    side_state = (x + side_dx, y + side_dy)
                    if 0 <= side_state[0] < width and 0 <= side_state[1] < height and side_state not in walls:
                        side_state_idx = xy_to_state(*side_state, width)
                        transition_matrix[current_state, action, side_state_idx] += 0.1
                    else:
                        transition_matrix[current_state, action, current_state] += 0.1

    for (x, y), reward in terminal_states.items():
        state_idx = xy_to_state(x, y, width)
        reward_matrix[:, :, state_idx] = reward

    return transition_matrix, reward_matrix

# Function to read and parse grid configuration from a file
def read_grid_config(file_path):
    terminal_states = {}
    walls = set()
    with open(file_path, 'r') as file:
        width, height = map(int, file.readline().split())
        immediate_reward = float(file.readline().strip())
        
        for line in file:
            parts = line.strip().split()
            if parts[0] == 'T':
                # Terminal state
                x, y, reward = int(parts[1]), int(parts[2]), float(parts[3])
                terminal_states[(x, y)] = reward
            elif parts[0] == 'W':
                # Wall
                x, y = int(parts[1]), int(parts[2])
                walls.add((x, y))

    return width, height, terminal_states, walls, immediate_reward

# Read grid configuration from file for Grid 1
grid1_config = read_grid_config("grid1.txt")
transition_matrix1, reward_matrix1 = create_matrices(*grid1_config)

# Example of how to access a specific transition matrix
print("Transition Matrix for Grid 1, State (0, 0):")
print(transition_matrix1[xy_to_state(0, 0, grid1_config[0]), :, :])


## Task 2 - Implement functions

let's start with implementing Value Iteration, Policy Iteration, TD-Learning, and Q-Learning for our Grid World environments. Each of these methods serves a different purpose in reinforcement learning:

Value Iteration and Policy Iteration are dynamic programming methods used for solving Markov Decision Processes (MDPs).
TD-Learning (Temporal Difference Learning) and Q-Learning are methods used in model-free reinforcement learning, where the agent learns from experience without knowledge of the environment's dynamics.
Let's begin with Value Iteration and Policy Iteration, followed by TD-Learning and Q-Learning implementations.

1. Value Iteration
Value Iteration is an iterative algorithm used to compute the optimal value function and derive the optimal policy. It repeatedly updates the value of each state until the values converge.

2. Policy Iteration
Policy Iteration consists of two steps: policy evaluation and policy improvement. In policy evaluation, the value function is computed for a given policy. In policy improvement, the policy is updated using the value function.

3. TD-Learning
TD-Learning is a combination of Monte Carlo ideas and dynamic programming ideas. It updates the value estimate based on the estimate of subsequent states.

4. Q-Learning
Q-Learning is an off-policy TD control algorithm. It learns the value of an action in a particular state and uses this to form a policy.

We'll start with the code for Value Iteration and Policy Iteration. Since these algorithms require complete knowledge of the environment (transition probabilities and rewards), we will use the matrices we created earlier.

Here's the implementation for Value Iteration and Policy Iteration:

In [7]:
import numpy as np

# Function to convert (x, y) coordinates to a state index
def xy_to_state(x, y, width):
    return y * width + x

# Function to create transition and reward matrices for a given grid configuration
def create_matrices(width, height, terminal_states, walls, immediate_reward):
    num_states = width * height
    num_actions = 4  # Up, Down, Left, Right
    transition_matrix = np.zeros((num_states, num_actions, num_states))
    reward_matrix = np.zeros((num_states, num_actions, num_states)) + immediate_reward

    for x in range(width):
        for y in range(height):
            if (x, y) in walls or (x, y) in terminal_states:
                continue

            current_state = xy_to_state(x, y, width)

            for action, (dx, dy) in enumerate([(-1, 0), (1, 0), (0, -1), (0, 1)]):
                next_state = (x + dx, y + dy)

                if 0 <= next_state[0] < width and 0 <= next_state[1] < height and next_state not in walls:
                    next_state_idx = xy_to_state(*next_state, width)
                    transition_matrix[current_state, action, next_state_idx] += 0.8

                for side_dx, side_dy in [(dy, dx), (-dy, -dx)]:
                    side_state = (x + side_dx, y + side_dy)
                    if 0 <= side_state[0] < width and 0 <= side_state[1] < height and side_state not in walls:
                        side_state_idx = xy_to_state(*side_state, width)
                        transition_matrix[current_state, action, side_state_idx] += 0.1
                    else:
                        transition_matrix[current_state, action, current_state] += 0.1

    for (x, y), reward in terminal_states.items():
        state_idx = xy_to_state(x, y, width)
        reward_matrix[:, :, state_idx] = reward

    return transition_matrix, reward_matrix

# Function to read and parse grid configuration from a file
def read_grid_config(file_path):
    terminal_states = {}
    walls = set()
    with open(file_path, 'r') as file:
        width, height = map(int, file.readline().split())
        immediate_reward = float(file.readline().strip())
        
        for line in file:
            parts = line.strip().split()
            if parts[0] == 'T':
                # Terminal state
                x, y, reward = int(parts[1]), int(parts[2]), float(parts[3])
                terminal_states[(x, y)] = reward
            elif parts[0] == 'W':
                # Wall
                x, y = int(parts[1]), int(parts[2])
                walls.add((x, y))

    return width, height, terminal_states, walls, immediate_reward

# Read grid configuration from file for Grid 1
grid1_config = read_grid_config("grid1.txt")
transition_matrix1, reward_matrix1 = create_matrices(*grid1_config)

# Example of how to access a specific transition matrix
print("Transition Matrix for Grid 1, State (0, 0):")
print(transition_matrix1[xy_to_state(0, 0, grid1_config[0]), :, :])

def value_iteration(transition_matrix, reward_matrix, gamma=0.9, threshold=0.01):
    num_states = transition_matrix.shape[0]
    num_actions = transition_matrix.shape[1]

    value = np.zeros(num_states)
    policy = np.zeros(num_states, dtype=int)

    while True:
        new_value = np.copy(value)
        for s in range(num_states):
            value_actions = np.sum(transition_matrix[s, :, :] * (reward_matrix[s, :, :] + gamma * value), axis=1)
            new_value[s] = np.max(value_actions)

        if np.max(np.abs(new_value - value)) < threshold:
            break

        value = new_value

    for s in range(num_states):
        policy[s] = np.argmax(np.sum(transition_matrix[s, :, :] * (reward_matrix[s, :, :] + gamma * value), axis=1))

    return policy, value

def policy_iteration(transition_matrix, reward_matrix, gamma=0.9, threshold=0.01):
    num_states = transition_matrix.shape[0]
    num_actions = transition_matrix.shape[1]

    policy = np.random.choice(num_actions, num_states)
    value = np.zeros(num_states)

    def policy_evaluation(policy, value):
        while True:
            new_value = np.copy(value)
            for s in range(num_states):
                a = policy[s]
                value_a = np.sum(transition_matrix[s, a, :] * (reward_matrix[s, a, :] + gamma * value))
                new_value[s] = value_a

            if np.max(np.abs(new_value - value)) < threshold:
                break

            value = new_value

        return value

    while True:
        value = policy_evaluation(policy, value)
        policy_stable = True

        for s in range(num_states):
            old_action = policy[s]
            new_action = np.argmax(np.sum(transition_matrix[s, :, :] * (reward_matrix[s, :, :] + gamma * value), axis=1))
            policy[s] = new_action
            if old_action != new_action:
                policy_stable = False

        if policy_stable:
            break

    return policy, value

# Example usage with Grid 1 matrices
policy_vi, value_vi = value_iteration(transition_matrix1, reward_matrix1)
policy_pi, value_pi = policy_iteration(transition_matrix1, reward_matrix1)

print("Value Iteration Policy:", policy_vi)
print("Policy Iteration Policy:", policy_pi)

def td_learning(transition_matrix, reward_matrix, initial_state, alpha=0.1, gamma=0.9, num_episodes=1000):
    num_states = transition_matrix.shape[0]
    V = np.zeros(num_states)

    for _ in range(num_episodes):
        state = initial_state
        while True:
            action = np.random.choice(np.arange(transition_matrix.shape[1]))
            next_state_probabilities = transition_matrix[state, action, :]
            next_state = np.random.choice(np.arange(num_states), p=next_state_probabilities)
            
            reward = reward_matrix[state, action, next_state]
            V[state] = V[state] + alpha * (reward + gamma * V[next_state] - V[state])
            
            if np.max(next_state_probabilities) == 1.0:  # Terminal state
                break
            state = next_state

    return V

def q_learning(transition_matrix, reward_matrix, alpha=0.1, gamma=0.9, epsilon=0.1, num_episodes=1000):
    num_states, num_actions = transition_matrix.shape[0], transition_matrix.shape[1]
    Q = np.zeros((num_states, num_actions))

    for _ in range(num_episodes):
        state = np.random.randint(num_states)
        
        while True:
            if np.random.uniform(0, 1) < epsilon:
                action = np.random.choice(num_actions)  # Explore
            else:
                action = np.argmax(Q[state, :])  # Exploit

            next_state_probabilities = transition_matrix[state, action, :]
            next_state = np.random.choice(np.arange(num_states), p=next_state_probabilities)
            
            reward = reward_matrix[state, action, next_state]
            Q[state, action] += alpha * (reward + gamma * np.max(Q[next_state, :]) - Q[state, action])

            if np.max(next_state_probabilities) == 1.0:  # Terminal state
                break
            state = next_state

    return Q

# Example usage with Grid 1 matrices
initial_state = 0  # Starting state for TD-Learning
V_td = td_learning(transition_matrix1, reward_matrix1, initial_state)
Q_qlearning = q_learning(transition_matrix1, reward_matrix1)

print("TD-Learning Value Function:", V_td)
print("Q-Learning Q-Table:", Q_qlearning)



## Task 3 - Experiments

For the three grids you have, perform the following analysis and include them in your report,.

- Compare value and policy iterations in terms of convergence time. Relate it to computational complexity, current implementation and number of iterations needed.

- How are optimal policies change with immediate reward values? Show some examples (similar to Figure 17.2b in AIMA)

- Compare TD-Learning and Q-Learning results with each other. Also with value and policy iterations. Remember that value and policy iteration solve Markov Decision Processes where we know the model (T and Rewards). TD-Learning and Q-Learning are (passive and active, respectively) Reinforcement Learning methods that don't have the model but use data from simulations. Data are simulated from the model we know, but the model is not used n TD or Q-Learning.

- What are the effects of epsilon and alpha values and how they are modified?

- What is the effect of number of episodes?

In [ ]:
import numpy as np

# Function to convert (x, y) coordinates to a state index
def xy_to_state(x, y, width):
    return y * width + x

# Function to create transition and reward matrices for a given grid configuration
def create_matrices(width, height, terminal_states, walls, immediate_reward):
    num_states = width * height
    num_actions = 4  # Up, Down, Left, Right
    transition_matrix = np.zeros((num_states, num_actions, num_states))
    reward_matrix = np.zeros((num_states, num_actions, num_states)) + immediate_reward

    for x in range(width):
        for y in range(height):
            if (x, y) in walls or (x, y) in terminal_states:
                continue

            current_state = xy_to_state(x, y, width)

            for action, (dx, dy) in enumerate([(-1, 0), (1, 0), (0, -1), (0, 1)]):
                next_state = (x + dx, y + dy)

                if 0 <= next_state[0] < width and 0 <= next_state[1] < height and next_state not in walls:
                    next_state_idx = xy_to_state(*next_state, width)
                    transition_matrix[current_state, action, next_state_idx] += 0.8

                for side_dx, side_dy in [(dy, dx), (-dy, -dx)]:
                    side_state = (x + side_dx, y + side_dy)
                    if 0 <= side_state[0] < width and 0 <= side_state[1] < height and side_state not in walls:
                        side_state_idx = xy_to_state(*side_state, width)
                        transition_matrix[current_state, action, side_state_idx] += 0.1
                    else:
                        transition_matrix[current_state, action, current_state] += 0.1

    for (x, y), reward in terminal_states.items():
        state_idx = xy_to_state(x, y, width)
        reward_matrix[:, :, state_idx] = reward

    return transition_matrix, reward_matrix

# Function to read and parse grid configuration from a file
def read_grid_config(file_path):
    terminal_states = {}
    walls = set()
    with open(file_path, 'r') as file:
        width, height = map(int, file.readline().split())
        immediate_reward = float(file.readline().strip())
        
        for line in file:
            parts = line.strip().split()
            if parts[0] == 'T':
                # Terminal state
                x, y, reward = int(parts[1]), int(parts[2]), float(parts[3])
                terminal_states[(x, y)] = reward
            elif parts[0] == 'W':
                # Wall
                x, y = int(parts[1]), int(parts[2])
                walls.add((x, y))

    return width, height, terminal_states, walls, immediate_reward

# Read grid configuration from file for Grid 1
grid1_config = read_grid_config("grid1.txt")
transition_matrix1, reward_matrix1 = create_matrices(*grid1_config)

# Example of how to access a specific transition matrix
print("Transition Matrix for Grid 1, State (0, 0):")
print(transition_matrix1[xy_to_state(0, 0, grid1_config[0]), :, :])

def value_iteration(transition_matrix, reward_matrix, gamma=0.9, threshold=0.01):
    num_states = transition_matrix.shape[0]
    num_actions = transition_matrix.shape[1]

    value = np.zeros(num_states)
    policy = np.zeros(num_states, dtype=int)

    while True:
        new_value = np.copy(value)
        for s in range(num_states):
            value_actions = np.sum(transition_matrix[s, :, :] * (reward_matrix[s, :, :] + gamma * value), axis=1)
            new_value[s] = np.max(value_actions)

        if np.max(np.abs(new_value - value)) < threshold:
            break

        value = new_value

    for s in range(num_states):
        policy[s] = np.argmax(np.sum(transition_matrix[s, :, :] * (reward_matrix[s, :, :] + gamma * value), axis=1))

    return policy, value

def policy_iteration(transition_matrix, reward_matrix, gamma=0.9, threshold=0.01):
    num_states = transition_matrix.shape[0]
    num_actions = transition_matrix.shape[1]

    policy = np.random.choice(num_actions, num_states)
    value = np.zeros(num_states)

    def policy_evaluation(policy, value):
        while True:
            new_value = np.copy(value)
            for s in range(num_states):
                a = policy[s]
                value_a = np.sum(transition_matrix[s, a, :] * (reward_matrix[s, a, :] + gamma * value))
                new_value[s] = value_a

            if np.max(np.abs(new_value - value)) < threshold:
                break

            value = new_value

        return value

    while True:
        value = policy_evaluation(policy, value)
        policy_stable = True

        for s in range(num_states):
            old_action = policy[s]
            new_action = np.argmax(np.sum(transition_matrix[s, :, :] * (reward_matrix[s, :, :] + gamma * value), axis=1))
            policy[s] = new_action
            if old_action != new_action:
                policy_stable = False

        if policy_stable:
            break

    return policy, value

# Example usage with Grid 1 matrices
policy_vi, value_vi = value_iteration(transition_matrix1, reward_matrix1)
policy_pi, value_pi = policy_iteration(transition_matrix1, reward_matrix1)

print("Value Iteration Policy:", policy_vi)
print("Policy Iteration Policy:", policy_pi)

def td_learning(transition_matrix, reward_matrix, initial_state, alpha=0.1, gamma=0.9, num_episodes=1000):
    num_states = transition_matrix.shape[0]
    V = np.zeros(num_states)

    for _ in range(num_episodes):
        state = initial_state
        while True:
            action = np.random.choice(np.arange(transition_matrix.shape[1]))
            next_state_probabilities = transition_matrix[state, action, :]
            next_state = np.random.choice(np.arange(num_states), p=next_state_probabilities)
            
            reward = reward_matrix[state, action, next_state]
            V[state] = V[state] + alpha * (reward + gamma * V[next_state] - V[state])
            
            if np.max(next_state_probabilities) == 1.0:  # Terminal state
                break
            state = next_state

    return V

def q_learning(transition_matrix, reward_matrix, alpha=0.1, gamma=0.9, epsilon=0.1, num_episodes=1000):
    num_states, num_actions = transition_matrix.shape[0], transition_matrix.shape[1]
    Q = np.zeros((num_states, num_actions))

    for _ in range(num_episodes):
        state = np.random.randint(num_states)
        
        while True:
            if np.random.uniform(0, 1) < epsilon:
                action = np.random.choice(num_actions)  # Explore
            else:
                action = np.argmax(Q[state, :])  # Exploit

            next_state_probabilities = transition_matrix[state, action, :]
            next_state = np.random.choice(np.arange(num_states), p=next_state_probabilities)
            
            reward = reward_matrix[state, action, next_state]
            Q[state, action] += alpha * (reward + gamma * np.max(Q[next_state, :]) - Q[state, action])

            if np.max(next_state_probabilities) == 1.0:  # Terminal state
                break
            state = next_state

    return Q

# Example usage with Grid 1 matrices
initial_state = 0  # Starting state for TD-Learning
V_td = td_learning(transition_matrix1, reward_matrix1, initial_state)
Q_qlearning = q_learning(transition_matrix1, reward_matrix1)

print("TD-Learning Value Function:", V_td)
print("Q-Learning Q-Table:", Q_qlearning)


## 1. Setup for Comparing Value and Policy Iterations
# We will measure and log the time taken for convergence and the number of iterations required.
import time

def run_value_policy_iterations(transition_matrix, reward_matrix):
    start_time = time.time()
    policy_vi, value_vi = value_iteration(transition_matrix, reward_matrix)
    vi_time = time.time() - start_time

    start_time = time.time()
    policy_pi, value_pi = policy_iteration(transition_matrix, reward_matrix)
    pi_time = time.time() - start_time

    return vi_time, pi_time

# Example usage
vi_time_grid1, pi_time_grid1 = run_value_policy_iterations(transition_matrix1, reward_matrix1)
print(f"Grid 1: Value Iteration Time: {vi_time_grid1}, Policy Iteration Time: {pi_time_grid1}")


## 2. Setup for Optimal Policies with Different Immediate Rewards
# We will modify the immediate rewards and log the resulting optimal policies.
def experiment_with_rewards(transition_matrix, original_reward_matrix, reward_values):
    policies = {}
    for reward in reward_values:
        modified_reward_matrix = np.copy(original_reward_matrix)
        modified_reward_matrix[modified_reward_matrix != 0] = reward  # Change non-zero rewards
        policy, _ = value_iteration(transition_matrix, modified_reward_matrix)
        policies[reward] = policy
    return policies

# Example usage
reward_values = [-0.02, -0.04, -0.06]
policies_grid1 = experiment_with_rewards(transition_matrix1, reward_matrix1, reward_values)
print(f"Grid 1 Policies with Different Rewards: {policies_grid1}")

## 3. Setup for Comparing TD-Learning and Q-Learning
# We will run TD-Learning and Q-Learning and log their results for comparison.

def run_td_q_learning(transition_matrix, reward_matrix, initial_state_td):
    V_td = td_learning(transition_matrix, reward_matrix, initial_state_td)
    Q_qlearning = q_learning(transition_matrix, reward_matrix)
    return V_td, Q_qlearning

# Example usage
V_td_grid1, Q_qlearning_grid1 = run_td_q_learning(transition_matrix1, reward_matrix1, initial_state=0)
print(f"Grid 1 TD-Learning Value Function: {V_td_grid1}")
print(f"Grid 1 Q-Learning Q-Table: {Q_qlearning_grid1}")

## 4. & 5. Effects of Epsilon, Alpha Values, and Number of Episodes
# We will vary epsilon, alpha, and the number of episodes, and log the Q-tables and value functions.

def run_q_learning_with_params(transition_matrix, reward_matrix, alphas, epsilons, episodes):
    results = {}
    for alpha in alphas:
        for epsilon in epsilons:
            for episode in episodes:
                Q = q_learning(transition_matrix, reward_matrix, alpha, gamma, epsilon, episode)
                results[(alpha, epsilon, episode)] = Q
    return results

# Example usage
alphas = [0.1, 0.5, 0.9]
epsilons = [0.1, 0.5, 0.9]
episodes = [500, 1000, 2000]
q_learning_results_grid1 = run_q_learning_with_params(transition_matrix1, reward_matrix1, alphas, epsilons, episodes)
print(f"Grid 1 Q-Learning Results with Varying Parameters: {q_learning_results_grid1}")


## Task 4 (extra credit) - What if diagonal moves were possible?

Repeat the experiments in one of these settings. You don't have to stick to the code provided here. But, you can only use Numpy (and matplotlib).

You should decide on how to distribute the noise associated with actions across (next) states. For example, in the scenario above we have probability of going Up with the Up action is 1-noise. The probability of going Left with the Up action is noise/2. The same for going Right.

Here's an approach to set up the experiments:

1. Adjust Transition Matrix: Modify the transition matrix creation to reflect the specified noise distribution.

2. Conduct Experiments: Use the adjusted transition matrix to run experiments for Value Iteration, Policy Iteration, TD-Learning, and Q-Learning.

3. Data Collection and Logging: Collect and log data for each experiment, including convergence times, policy changes, and the effects of various parameters.

4. Visualization: Optionally, use Matplotlib to visualize the results, like policy changes or learning curves.

### Conduct Experiments
Now, use the transition_matrix_noise_grid1 along with the reward matrix to run the experiments for each algorithm. You can use the functions defined in the previous messages for Value Iteration, Policy Iteration, TD-Learning, and Q-Learning.

### Data Collection and Logging
After running each experiment, collect data such as the time taken for convergence, the policies obtained, and the value functions. Log these details for analysis.

### Visualization with Matplotlib
If you want to visualize the results, you can use Matplotlib. For instance, you can plot the value function over iterations or show how the optimal policy changes with different immediate rewards.

In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt

# Function to read grid configuration from a file
def read_grid_config(file_path):
    terminal_states = {}
    walls = set()
    with open(file_path, 'r') as file:
        width, height = map(int, file.readline().split())
        immediate_reward = float(file.readline().strip())
        
        for line in file:
            parts = line.strip().split()
            if parts[0] == 'T':
                # Terminal state
                x, y, reward = int(parts[1]), int(parts[2]), float(parts[3])
                terminal_states[(x, y)] = reward
            elif parts[0] == 'W':
                # Wall
                x, y = int(parts[1]), int(parts[2])
                walls.add((x, y))

    return width, height, terminal_states, walls, immediate_reward

# Function to convert (x, y) coordinates to a state index
def xy_to_state(x, y, width):
    return y * width + x

# Function to create transition matrix with specified noise
def create_transition_matrix_with_noise(width, height, walls, terminal_states, noise):
    num_states = width * height
    num_actions = 4  # Up, Down, Left, Right
    transition_matrix = np.zeros((num_states, num_actions, num_states))

    for x in range(width):
        for y in range(height):
            if (x, y) in walls or (x, y) in terminal_states:
                continue

            current_state = xy_to_state(x, y, width)

            for action, (dx, dy) in enumerate([(-1, 0), (1, 0), (0, -1), (0, 1)]):
                # Calculate next state for the intended action
                next_state = (x + dx, y + dy)
                if 0 <= next_state[0] < width and 0 <= next_state[1] < height and next_state not in walls:
                    next_state_idx = xy_to_state(*next_state, width)
                    transition_matrix[current_state, action, next_state_idx] += 1 - noise

                # Calculate next states for unintended actions (noise)
                for side_dx, side_dy in [(dy, dx), (-dy, -dx)]:
                    side_state = (x + side_dx, y + side_dy)
                    if 0 <= side_state[0] < width and 0 <= side_state[1] < height and side_state not in walls:
                        side_state_idx = xy_to_state(*side_state, width)
                        transition_matrix[current_state, action, side_state_idx] += noise / 2
                    else:
                        transition_matrix[current_state, action, current_state] += noise / 2

    return transition_matrix

# Read grid configuration from file
grid1_config = read_grid_config("grid1.txt")
width1, height1, terminal_states1, walls1, immediate_reward1 = grid1_config

# Create transition and reward matrices for Grid 1 with noise
noise_grid1 = 0.2
transition_matrix_noise_grid1 = create_transition_matrix_with_noise(width1, height1, walls1, terminal_states1, noise_grid1)
reward_matrix1 = np.zeros((width1 * height1, 4, width1 * height1)) + immediate_reward1
for terminal_state, reward in terminal_states1.items():
    reward_matrix1[xy_to_state(*terminal_state, width1), :, xy_to_state(*terminal_state, width1)] = reward

# Now you can proceed to run the experiments as discussed in the previous steps.


# Example: Plotting the value function over iterations
values = [value_iteration(transition_matrix_noise_grid1, reward_matrix1, threshold=0.01)[1] for _ in range(10)]
plt.plot(values)
plt.xlabel('Iterations')
plt.ylabel('Value Function')
plt.title('Value Function over Iterations')
plt.show()

